In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import datasets, transforms
#from capsnet import CapsNet
#from data_loader import Dataset
from tqdm import tqdm
import time
import math
import matplotlib.pyplot as plt

from itertools import combinations, permutations
import time
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
import math
import itertools
import matplotlib.cm as cm
import subprocess
import os

from shutil import copyfile
from functools import partial

from scipy.special import gammaln

#import pathos.multiprocessing as mp

from scipy.interpolate import interp1d

import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
#import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.utils.model_zoo as model_zoo

#import pathos.multiprocessing as mp
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier


from functools import partial
import pathos.multiprocessing as mp

import sys
import tqdm

In [ ]:
def significance(s, b):
    if(b!=0):
        square_s = s/math.sqrt(b)
        if(square_s>0):
            return square_s
        else: return 0
    if(b==0 and s==0):
        return 0
    if(b==0 and s!=0):
        print (" a strange case")
        return s

In [ ]:
class ResNet(nn.Module):

    def __init__(self, in_channel=1, kin_features=21, num_classes=2):
        super(ResNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv3d(in_channel, 32, kernel_size=(3,3,3), stride=(1,1,1), padding=(0,1,1), bias=True),
            nn.BatchNorm3d(32),
            #nn.ReLU(inplace=True)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=(3,3), stride=(1,1), padding=1, bias=True),
            nn.BatchNorm2d(32),
            #nn.ReLU(inplace=True)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=(3,3), stride=(2,2), padding=1, bias=True),
            nn.BatchNorm2d(32),
            #nn.ReLU(inplace=True)
        )
        self.relu = nn.ReLU(inplace=True)
        
        #self.fc = nn.Linear(512 * block.expansion, num_classes)
        self.fc1 = nn.Linear(32*4*4, 1500)
        #self.fc2 = nn.Linear(1500+600, num_classes)
        
        self.kin_fc = nn.Sequential(
            nn.Linear(kin_features, 600),
            #nn.BatchNorm1d(2**8),
            nn.ReLU(),
            nn.Linear(600, 600),
            nn.ReLU(),
            nn.Linear(600, 600),
            nn.ReLU(),
            nn.Linear(600, 600),
            nn.ReLU(),
            nn.Linear(600, 600),
            nn.ReLU(),
            #nn.Tanh()
        )
        self.fc2 = nn.Sequential(
            nn.Linear(1500+600, 600),
            #nn.BatchNorm1d(2**8),
            nn.ReLU(),
            nn.Linear(600, 600),
            nn.ReLU(),
            nn.Linear(600, 600),
            nn.ReLU(),
            nn.Linear(600, 600),
            nn.ReLU(),
            nn.Linear(600, num_classes)
            #nn.ReLU(),
            #nn.Tanh()
        )



    def _forward_impl(self, x):
        c1 = self.conv1(x.unsqueeze(1))
        c1 = self.relu(c1)
        c1 = c1.squeeze()
        c2 = self.conv2(c1)
        c2 = self.relu(c2)
        c3 = self.conv2(c2)
        b2 = self.conv2(c1)
        b2 = self.relu(b2)
        b3 = self.conv2(b2)
        b3 = self.relu(b3)
        b4 = self.conv2(b3)
        #print (c1.size(), c3.size(), b4.size())
        c10 = c1+c3+b4
        c10 = self.relu(c10)
        
        c11 = self.conv3(c10)
        c11 = self.relu(c11)
        c12 = self.conv2(c11)
        c13 = self.conv3(c10)
        c14 = c12+c13
        c14 = self.relu(c14)
        
        c15 = self.conv2(c14)
        c15 = self.relu(c15)
        c16 = self.conv2(c15)
        c17 = c14+c16
        c17 = self.relu(c17)
        
        c18 = self.conv2(c17)
        c18 = self.relu(c18)
        c19 = self.conv2(c18)
        c20 = c17+c19
        c20 = self.relu(c20)
        
        c21 = self.conv3(c20)
        c21 = self.relu(c21)
        c22 = self.conv2(c21)
        c23 = self.conv3(c20)
        c24 = c22+c23
        c24 = self.relu(c24)
        
        c25 = self.conv2(c24)
        c25 = self.relu(c25)
        c26 = self.conv2(c25)
        c27 = c24+c26
        c27 = self.relu(c27)
        
        c28 = self.conv2(c27)
        c28 = self.relu(c28)
        c29 = self.conv2(c28)
        c30 = c27+c29
        c30 = self.relu(c30)

        c31 = self.conv3(c30)
        c31 = self.relu(c31)
        c32 = self.conv2(c31)
        c33 = self.conv3(c30)
        c34 = c32+c33
        c34 = self.relu(c34)
        
        c35 = self.conv2(c34)
        c35 = self.relu(c35)
        c36 = self.conv2(c35)
        c37 = c34+c36
        c37 = self.relu(c37)

        c38 = self.conv2(c37)
        c38 = self.relu(c38)
        c39 = self.conv2(c38)
        c40 = c37+c39
        c40 = self.relu(c40)

        c41 = self.conv3(c40)
        c41 = self.relu(c41)
        c42 = self.conv2(c41)
        c43 = self.conv3(c40)
        c44 = c42+c43
        c44 = self.relu(c44)
    
        return c44

    def forward(self, x, data):
        x = self._forward_impl(x)
        #print ("x", x.size())
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        data = self.kin_fc(data)
        out = torch.cat((x, data), dim=1)
        out = self.fc2(out)
        return F.softmax(out)
        
        #return self._forward_impl(x, data)


In [ ]:
%%time
train_data = np.load("./v13_combo_train.npy").astype(np.float16)
test_data = np.load("./v13_combo_test.npy").astype(np.float16)
train_data_V2 = np.load("./v13_kinematic_combo_train.npy").astype(np.float32)
test_data_V2 = np.load("./v13_kinematic_combo_test.npy").astype(np.float32)



In [ ]:
Y_train = np.concatenate((np.zeros(190000), np.ones(99000), np.ones(55000), np.ones(2000), np.ones(2500), np.ones(10000), np.ones(600))).astype(np.int)
Y_test = np.concatenate((np.zeros(50000), np.ones(190000), np.ones(8000), np.ones(1000), np.ones(1000), np.ones(2400), np.ones(100))).astype(np.int)


In [ ]:
GPU_NUM=0
TRAIN_ON_GPU = torch.cuda.is_available()
torch.cuda.empty_cache()
torch.backends.cudnn.deterministic=True

cuda = torch.device(f"cuda:{GPU_NUM}" if TRAIN_ON_GPU else "cpu")
torch.cuda.set_device(cuda)

print(cuda)

In [ ]:
np.random.seed(0)
idx = np.arange(len(train_data))
np.random.shuffle(idx)
train_data, Y_train = train_data[idx], Y_train[idx]
train_data_V2 = train_data_V2[idx]

In [ ]:
start_time = time.time()

cnn = ResNet(in_channel=1, kin_features=21, num_classes=2).float().cuda()

cnn = nn.DataParallel(cnn)

#optimizer = torch.optim.Adam(capsule_net.parameters())

num_epochs = 30

result = []
# Loss and Optimizer
criterion = nn.CrossEntropyLoss(weight=torch.Tensor([1.0, 1.0]).float()).cuda()

learning_rate = 1e-4
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate, weight_decay=0.0)
#scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=math.exp(-0.0))
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=math.exp(-0.01))  
rl_ = []
# Train the Model
rloss_, eloss_ = [], []
racc_, eacc_ = [], []
for epoch in range(num_epochs):
    #np.random.seed(0)
    idx = np.arange(len(train_data))
    np.random.shuffle(idx)
    x_train, y_train = train_data[idx], Y_train[idx]
    x_train_V2 = train_data_V2[idx]

    x_train_split = np.array_split(x_train, 500)
    x_train_V2_split = np.array_split(x_train_V2, 500)
    y_train_split = np.array_split(y_train, 500)

    x_test_split = np.array_split(test_data, 500)
    x_test_V2_split = np.array_split(test_data_V2, 500)
    y_test_split = np.array_split(Y_test, 500)

    train_loss, train_correct = 0, 0
    for i in range(len(x_train_split)):
        x_train = x_train_split[i]
        x_train_V2 = x_train_V2_split[i]
        y_train = y_train_split[i]
        x_train = Variable(torch.from_numpy(x_train)).float().cuda()
        x_train_V2 = Variable(torch.from_numpy(x_train_V2)).float().cuda()
        y_train = Variable(torch.from_numpy(y_train)).long().cuda()


        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(x_train, x_train_V2)
        pred = outputs.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct = pred.eq(y_train.data.view_as(pred)).long().sum().item()
        train_correct+=correct

        loss = criterion(outputs, y_train)
        #print (i, loss.data.item())
        train_loss+=loss.data.item()
        loss.backward()
        optimizer.step()
    rl_.append(scheduler.get_lr()[0])
    scheduler.step()

    #if(epoch<=10 or epoch%10==0):
    if(True):
        torch.cuda.empty_cache() #release some memory
        cnn = cnn.eval()
        
        test_result = []
        test_loss, test_correct = 0, 0
        for i in range(len(x_test_split)):
            x_test = x_test_split[i]
            x_test_V2 = x_test_V2_split[i]
            y_test = y_test_split[i]
            x_test = Variable(torch.from_numpy(x_test)).float().cuda()
            x_test_V2 = Variable(torch.from_numpy(x_test_V2)).float().cuda()
            y_test = Variable(torch.from_numpy(y_test)).long().cuda()

            outputs = cnn(x_test, x_test_V2)
            pred = outputs.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct = pred.eq(y_test.data.view_as(pred)).long().sum().item()
            test_correct+=correct

            loss = criterion(outputs, y_test)
            test_loss+=loss.data.item()
            
            test_result.append(outputs[:,0].detach().cpu().numpy().reshape(1, -1)[0])
            
        cnn = cnn.train()
        
        test_result = np.concatenate((test_result))
        #signal, tt, tw = test_result[:50000], test_result[50000:50000+170000], test_result[50000+170000:]
        np.save("./test_result"+str(epoch), test_result)
        
        plt.hist(test_result[:50000], bins=np.linspace(0.0, 1.0, 100), color='r', label="Signal", density=True, alpha=0.4)
        plt.hist(test_result[50000:50000+190000], bins=np.linspace(0.0, 1.0, 100), color='b', label="tt", density=True, alpha=0.4)
        plt.hist(test_result[50000+190000:50000+190000+8000], bins=np.linspace(0.0, 1.0, 100), label="tw", density=True, alpha=0.4)
        plt.hist(test_result[50000+190000+8000:50000+190000+8000+1000], bins=np.linspace(0.0, 1.0, 100), label="tth", density=True, alpha=0.4)
        plt.hist(test_result[50000+190000+8000+1000:50000+190000+8000+1000+1000], bins=np.linspace(0.0, 1.0, 100), label="ttv", density=True, alpha=0.4)
        plt.hist(test_result[50000+190000+8000+1000+1000:50000+190000+8000+1000+1000+2400], bins=np.linspace(0.0, 1.0, 100), label="llbj", density=True, alpha=0.4)
        plt.hist(test_result[50000+190000+8000+1000+1000+2400:50000+190000+8000+1000+1000+2400+100], bins=np.linspace(0.0, 1.0, 100), label="tatabb", density=True, alpha=0.4)
        plt.xlabel("NN Output", fontsize=15)
        plt.ylabel("Number", fontsize=15)
        plt.legend(loc='best', fontsize=15)
        plt.show()

        
        signal, ax = np.histogram(test_result[:50000], bins=np.linspace(0.99999, 1.0, 10000))
        tt, _ = np.histogram(test_result[50000:50000+190000], bins=np.linspace(0.99999, 1.0, 10000))
        tw, _ = np.histogram(test_result[50000+190000:50000+190000+8000], bins=np.linspace(0.99999, 1.0, 10000))
        tth, _ = np.histogram(test_result[50000+190000+8000:50000+190000+8000+1000], bins=np.linspace(0.99999, 1.0, 10000))
        ttv, _ = np.histogram(test_result[50000+190000+8000+1000:50000+190000+8000+1000+1000], bins=np.linspace(0.99999, 1.0, 10000))
        llbj, _ = np.histogram(test_result[50000+190000+8000+1000+1000:50000+190000+8000+1000+1000+2400], bins=np.linspace(0.99999, 1.0, 10000))
        tatabb, _ = np.histogram(test_result[50000+190000+8000+1000+1000+2400:50000+190000+8000+1000+1000+2400+100], bins=np.linspace(0.99999, 1.0, 10000))

        signal, tt, tw, tth, ttv, llbj, tatabb = signal*3e3*0.0214964/50000, tt*3e3*120.907/190000, tw*3e3*4.38354/8000, tth*3e3*0.15258/1000, ttv*3e3*0.157968/1000, llbj*3e3*1.22936/2400, tatabb*3e3*0.011392/100
        
        significance_ = [significance(sum(signal[idx:]), sum(tt[idx:])+sum(tw[idx:])+sum(tth[idx:])+sum(ttv[idx:])+sum(llbj[idx:])+sum(tatabb[idx:])) for idx in range(len(signal))]
        sig_number = [sum(signal[idx:]) for idx in range(len(signal))]
        
         
        plt.plot(sig_number, significance_, color='r', label="Significance")
        plt.xlabel("Signal Number", fontsize=15)
        plt.ylabel("Significance", fontsize=15)
        plt.legend(loc='best', fontsize=15)
        plt.show()

        print ("Epoch: %d, Time: %.2f, Lr: %.5f, Max Sig: %.6f, signal: %.2f, tt: %.2f, tw: %.2f, Train Loss: %.4f" %(epoch, (time.time()-start_time)/3600., scheduler.get_lr()[0], np.max(significance_), sum(signal[np.argmax(significance_):]), sum(tt[np.argmax(significance_):]), sum(tw[np.argmax(significance_):]), train_loss/len(x_train_split)))
        print ("Train Correct: %d, Test Correct: %d, Train Loss: %.4f, Test Loss: %.4f" %(train_correct, test_correct, train_loss/2000., test_loss/2000.))


